A simple 2D classifier

In [ ]:
# -*- coding: utf-8 -*-
import os, re, json, gc, sys, ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optional (not used in plotting)
import seaborn as sns  # noqa: F401
from scipy.signal import find_peaks  # noqa: F401
from scipy.signal import savgol_filter  # noqa: F401
from scipy.ndimage import gaussian_filter1d  # noqa: F401
from scipy.linalg import svd  # noqa: F401
from scipy import stats  # <-- NEW: t-test

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers
from sklearn.model_selection import KFold

# ----------------------------
# Repro & GPU memory growth
# ----------------------------
np.random.seed(42)
tf.random.set_seed(42)
try:
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for _gpu in gpus:
        tf.config.experimental.set_memory_growth(_gpu, True)
except Exception:
    pass

def hard_free():
    """Aggressively release memory after each bin."""
    try: plt.close('all')
    except: pass
    try: tf.keras.backend.clear_session()
    except: pass
    try: gc.collect(); gc.collect()
    except: pass
    try:
        import ctypes, platform
        if platform.system().lower() == "linux":
            ctypes.CDLL("libc.so.6").malloc_trim(0)
    except: pass

# ----------------------------
# Grouped log-odds gradient
# ----------------------------
@tf.function(reduce_retracing=True)
def _group_logodds_grad_for_model(x1, model, pos_ids, neg_ids, eps):
    pos_ids = tf.constant(pos_ids, dtype=tf.int32)
    neg_ids = tf.constant(neg_ids, dtype=tf.int32)
    with tf.GradientTape() as tape:
        tape.watch(x1)
        p = model(x1, training=False)  # (1, C)
        p_pos = tf.reduce_sum(tf.gather(p, pos_ids, axis=1), axis=1)  # (1,)
        p_neg = tf.reduce_sum(tf.gather(p, neg_ids, axis=1), axis=1)  # (1,)
        log_odds = tf.math.log(p_pos + eps) - tf.math.log(p_neg + eps)
    g = tape.gradient(log_odds, x1)  # (1, D)
    return tf.squeeze(g, axis=0)     # (D,)

def compute_avg_group_logodds_gradient(X: np.ndarray, models: list, pos_ids=(2,3), neg_ids=(0,1), eps: float = 1e-8) -> np.ndarray:
    X_t = tf.convert_to_tensor(X, dtype=tf.float32)
    N = int(X_t.shape[0])
    sample_grads = []
    for i in range(N):
        x_i = X_t[i:i+1]
        grads_over_models = []
        for m in models:
            g = _group_logodds_grad_for_model(x_i, m, pos_ids, neg_ids, eps)
            grads_over_models.append(g)
        g_avg_models = tf.reduce_mean(tf.stack(grads_over_models, axis=0), axis=0)
        sample_grads.append(g_avg_models)
    avg_grad = tf.reduce_mean(tf.stack(sample_grads, axis=0), axis=0)
    return avg_grad.numpy()

# ----------------------------
# Model
# ----------------------------
def build_model(input_dim: int, num_classes: int):
    model = Sequential([
        Dense(128, input_dim=input_dim, activation='relu', kernel_regularizer=regularizers.l1(0.01)),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])
    return model

# ----------------------------
# Helpers (cosine + plotting)
# ----------------------------
def cosine_sim(a: np.ndarray, b: np.ndarray, eps: float = 1e-12) -> float:
    a = np.asarray(a, dtype=float).ravel()
    b = np.asarray(b, dtype=float).ravel()
    n = min(a.size, b.size)
    a = a[:n]; b = b[:n]
    denom = (np.linalg.norm(a) * np.linalg.norm(b)) + eps
    return float(np.dot(a, b) / denom)

def mirror_plot(x, top_y, bottom_y, title, outfile):
    plt.figure(figsize=(10, 5))
    plt.plot(x, top_y, linewidth=1.0, label="Run A")
    plt.plot(x, -bottom_y, linewidth=1.0, label="Run B (mirrored)")
    plt.axhline(0.0, linewidth=0.8)
    plt.xlabel("m/z (approx grid)")
    plt.ylabel("Gradient magnitude")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.savefig(outfile, dpi=200)
    plt.close()

# ----------------------------
# NEW: Stats & plotting helpers for volcano + signed agreement
# ----------------------------
def ttest_and_log2fc(X: np.ndarray, y: np.ndarray, pos_ids, neg_ids, eps: float = 1e-12):
    """Welch's t-test (pos vs neg) per feature + log2FC (pos/neg)."""
    pos_mask = np.isin(y, list(pos_ids))
    neg_mask = np.isin(y, list(neg_ids))
    Xp = X[pos_mask]; Xn = X[neg_mask]
    # Handle edge cases gracefully
    if Xp.shape[0] < 2 or Xn.shape[0] < 2:
        # fall back to NaNs so plots show empty
        pvals = np.full(X.shape[1], np.nan)
        log2fc = np.full(X.shape[1], np.nan)
        return pvals, log2fc

    # Welch's t-test, axis=0 over features
    tstat, pvals = stats.ttest_ind(Xp, Xn, axis=0, equal_var=False, nan_policy='omit')
    mu_p = np.nanmean(Xp, axis=0)
    mu_n = np.nanmean(Xn, axis=0)
    log2fc = np.log2((mu_p + eps) / (mu_n + eps))
    # Replace any nan/inf from degenerate features
    pvals = np.where(np.isfinite(pvals), pvals, 1.0)
    log2fc = np.where(np.isfinite(log2fc), log2fc, 0.0)
    return pvals, log2fc

def volcano_plot(log2fc: np.ndarray, pvals: np.ndarray,
                 lfc_left: float, lfc_right: float, alpha: float,
                 title: str, outfile: str):
    xl = log2fc
    yl = -np.log10(np.clip(pvals, 1e-300, 1.0))
    sig_up   = (xl >= lfc_right) & (pvals <= alpha)
    sig_down = (xl <= lfc_left)  & (pvals <= alpha)
    other    = ~(sig_up | sig_down)

    plt.figure(figsize=(10,6))
    plt.scatter(xl[other], yl[other], s=15, alpha=0.6, color='lightgray')
    plt.scatter(xl[sig_down], yl[sig_down], s=30, alpha=0.9, color='C0')  # blue
    plt.scatter(xl[sig_up], yl[sig_up], s=30, alpha=0.9, color='C3')      # red
    # thresholds
    plt.axvline(lfc_left,  ls='--', lw=1, color='k')
    plt.axvline(lfc_right, ls='--', lw=1, color='k')
    plt.axhline(-np.log10(alpha), ls='--', lw=1, color='k')
    plt.xlabel("log2 fold-change (pos / neg)")
    plt.ylabel(r"$-\log_{10}(p)$")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(outfile, dpi=200)
    plt.close()

def signed_agreement_plot(log2fc: np.ndarray, pvals: np.ndarray, grad_mean: np.ndarray,
                          lfc_left: float, lfc_right: float, alpha: float,
                          title_prefix: str, outfile: str):
    signed_stat = log2fc * (-np.log10(np.clip(pvals, 1e-300, 1.0)))
    # Pearson r (ignore NaNs)
    mask = np.isfinite(signed_stat) & np.isfinite(grad_mean)
    if np.sum(mask) >= 2:
        r = np.corrcoef(signed_stat[mask], grad_mean[mask])[0,1]
    else:
        r = np.nan

    sig_up   = (log2fc >= lfc_right) & (pvals <= alpha)
    sig_down = (log2fc <= lfc_left)  & (pvals <= alpha)
    other    = ~(sig_up | sig_down)

    plt.figure(figsize=(10,6))
    plt.scatter(signed_stat[other], grad_mean[other], s=15, alpha=0.6, color='lightgray')
    plt.scatter(signed_stat[sig_down], grad_mean[sig_down], s=30, alpha=0.9, color='C0')
    plt.scatter(signed_stat[sig_up],   grad_mean[sig_up],   s=30, alpha=0.9, color='C3')
    plt.axvline(0.0, ls='--', lw=1, color='k'); plt.axhline(0.0, ls='--', lw=1, color='k')
    plt.xlabel(r"log2FC × $-\log_{10}(p)$  (signed)")
    plt.ylabel("Average log-odds gradient (RunA/B mean)")
    plt.title(f"{title_prefix}  (Pearson r = {r:.3f})")
    plt.tight_layout()
    plt.savefig(outfile, dpi=200)
    plt.close()
    return float(r)

# ----------------------------
# Splitter (4 CSVs per combined CSV) with bin prefix
# ----------------------------
TARGET_COLS = ["pos_runA", "pos_runB", "negabs_runA", "negabs_runB"]
MZ_COL = "m/z"

def split_csv(input_path: str, out_dir: str, bin_value: int) -> list[str]:
    """Split one combined CSV into 4 CSVs, prefixing filenames with bin number."""
    df = pd.read_csv(input_path)
    if MZ_COL not in df.columns:
        print(f"[SKIP] {input_path} (no '{MZ_COL}' column)")
        return []
    available_targets = [c for c in TARGET_COLS if c in df.columns]
    if not available_targets:
        print(f"[SKIP] {input_path} (none of {TARGET_COLS} found)")
        return []
    base = os.path.splitext(os.path.basename(input_path))[0]
    written = []
    for col in available_targets:
        out_df = df[[MZ_COL, col]].copy()
        out_path = os.path.join(out_dir, f"bin{bin_value}_{base}_{col}.csv")
        out_df.to_csv(out_path, index=False)
        written.append(out_path)
    return written

def process_folder(folder_path: str, bin_value: int) -> list[str]:
    """Split all CSVs in a folder to 'result/' and return list of result paths."""
    if not os.path.isdir(folder_path):
        print(f"[WARN] Not a folder: {folder_path}")
        return []
    out_dir = os.path.join(folder_path, "result")
    os.makedirs(out_dir, exist_ok=True)

    all_outputs = []
    for fname in os.listdir(folder_path):
        if fname.lower().endswith(".csv"):
            fpath = os.path.join(folder_path, fname)
            print(f"Splitting {fpath} ...")
            outputs = split_csv(fpath, out_dir, bin_value)
            all_outputs.extend(outputs)

    print(f"Split done. Wrote {len(all_outputs)} files to {out_dir}")
    return all_outputs

# ----------------------------
# Config (edit these)
# ----------------------------
CSV_PATH    = r"F:/20251010/dataset_rt_batch.csv"   # input dataset with 'bin' and 'target'
EPOCHS      = 50
BATCH_SIZE  = 32
K_SPLITS    = 5
N_REPEATS   = 2
SEED_BASES  = [111, 777]                   # two independent runs
OUT_ROOT    = r"F:/20251018/"              # everything goes directly under bin_<N>/
BIN_WHITELIST = None  # e.g. [35, 75]

# Volcano thresholds (edit to taste)
ALPHA = 0.05
LFC_LEFT, LFC_RIGHT = -0.5, 0.5

# Fixed grouping list (no prompt, no AUTO)
GROUPINGS = [((2,),(1,)), ((2,),(0,)), ((1,),(0,)), ((3,),(2,)), ((3,),(0,1,2)), ((2,1),(0,)), ((3,),(1,)),  ((3,),(0,))]

# ----------------------------
# KFold+Repeats trainer
# ----------------------------
def train_kfold_repeats(X: np.ndarray, Y: np.ndarray, seed_base: int):
    kf = KFold(n_splits=K_SPLITS, shuffle=True, random_state=42)
    all_models = []
    num_classes = int(np.max(Y)) + 1
    for fold, (tr, va) in enumerate(kf.split(X, Y), 1):
        X_tr, y_tr = X[tr], Y[tr]
        X_va, y_va = X[va], Y[va]
        for r in range(N_REPEATS):
            seed = seed_base * 1000 + fold * 100 + r
            tf.keras.utils.set_random_seed(seed)
            np.random.seed(seed)
            m = build_model(X.shape[1], num_classes)
            m.fit(X_tr, y_tr, epochs=EPOCHS, batch_size=BATCH_SIZE,
                  validation_data=(X_va, y_va), verbose=0)
            all_models.append(m)
        print(f"[Seed base {seed_base}] Fold {fold}/{K_SPLITS} trained {N_REPEATS} models (total: {len(all_models)})")
    return all_models

# ----------------------------
# MAIN (bin-root outputs + keep only split CSVs)
# ----------------------------
def main():
    df = pd.read_csv(CSV_PATH)
    # Discover bins
    bins_found = sorted([b for b in df["bin"].dropna().unique().tolist()])
    if BIN_WHITELIST is not None:
        bins_to_process = [b for b in bins_found if b in set(BIN_WHITELIST)]
    else:
        bins_to_process = bins_found
    if len(bins_to_process) == 0:
        raise ValueError(f"No 'bin' values found in {CSV_PATH}")

    # Keep only classes 0..3 by default
    df = df[df["target"].astype(int).isin([0,1,2,3])].copy()
    unique_labels = np.sort(df["target"].astype(int).unique())
    assert unique_labels[0] == 0 and np.array_equal(unique_labels, np.arange(unique_labels[-1] + 1)), \
        f"Non-contiguous labels detected: {unique_labels}. Please remap to 0..C-1."
    groupings = GROUPINGS
    print(f"\nUsing fixed grouping(s): {groupings}")

    os.makedirs(OUT_ROOT, exist_ok=True)

    for BIN_VALUE in bins_to_process:
        print(f"\n================= BIN {BIN_VALUE} =================")
        models_A = models_B = None
        X = Y = fdf = None

        try:
            fdf = df[df["bin"] == BIN_VALUE].copy()
            if fdf.empty:
                print(f"[WARN] No rows for bin {BIN_VALUE}; skipping.")
                continue

            # Normalize all features except ['bin','target'] within this bin
            cols_to_norm = fdf.columns.difference(['bin', 'target'])
            fdf[cols_to_norm] = fdf[cols_to_norm].apply(lambda x: x / (x.max() + 1.0))

            Y = fdf["target"].astype(int).to_numpy()
            feature_cols = fdf.columns.difference(['bin', 'target'])
            X = np.nan_to_num(fdf[feature_cols].to_numpy(), copy=False).astype(np.float32)

            if X.shape[0] < 2 or X.shape[1] < 1:
                print(f"[WARN] Insufficient data for bin {BIN_VALUE} (samples={X.shape[0]}, dim={X.shape[1]}). Skipping.")
                continue

            print(f"Bin {BIN_VALUE}: samples={X.shape[0]}, dim={X.shape[1]}  class_counts="
                  f"{dict(zip(*np.unique(Y, return_counts=True)))}")

            # Output dirs per bin (everything under bin folder)
            OUT_DIR   = os.path.join(OUT_ROOT, f"bin_{str(BIN_VALUE).replace('.', '_')}")
            CSV_DIR   = os.path.join(OUT_DIR, "csv")
            PLOTS_DIR = os.path.join(OUT_DIR, "plots")
            os.makedirs(OUT_DIR, exist_ok=True)
            os.makedirs(CSV_DIR, exist_ok=True)
            os.makedirs(PLOTS_DIR, exist_ok=True)

            # Train ensembles ONCE per run (A, B)
            models_A = train_kfold_repeats(X, Y, seed_base=SEED_BASES[0])
            models_B = train_kfold_repeats(X, Y, seed_base=SEED_BASES[1])

            # grid helper (for your mirror plots)
            def _make_grid(n):
                n_grid = min(10000, n)
                x_grid = np.arange(600, 600 + 0.1 * n_grid, 0.1)[:n_grid]
                return n_grid, x_grid

            def save_compare_only(pos_ids, neg_ids):
                # ---- Gradients (Run A & B)
                grad_A = compute_avg_group_logodds_gradient(X, models_A, pos_ids=pos_ids, neg_ids=neg_ids, eps=1e-8)
                grad_B = compute_avg_group_logodds_gradient(X, models_B, pos_ids=pos_ids, neg_ids=neg_ids, eps=1e-8)
                grad_mean = (grad_A + grad_B) / 2.0

                pos_tag = "_".join(map(str, pos_ids))
                neg_tag = "_".join(map(str, neg_ids))
                tag = f"pos_{pos_tag}__neg_{neg_tag}"

                # ---- Mirror plots (unchanged from your flow)
                n_grid, x_grid = _make_grid(min(grad_A.size, grad_B.size))
                yA = grad_A[:n_grid]; yB = grad_B[:n_grid]
                yA_pos = np.where(yA > 0, yA, 0.0)
                yB_pos = np.where(yB > 0, yB, 0.0)
                yA_neg = np.where(yA < 0, -yA, 0.0)  # abs
                yB_neg = np.where(yB < 0, -yB, 0.0)

                cos_pos = cosine_sim(yA_pos, yB_pos)
                cos_neg = cosine_sim(yA_neg, yB_neg)

                comb_csv = os.path.join(CSV_DIR, f"grads_AB__{tag}.csv")
                pd.DataFrame({
                    "m/z": x_grid,
                    "grad_runA": yA,
                    "grad_runB": yB,
                    "pos_runA": yA_pos,
                    "pos_runB": yB_pos,
                    "negabs_runA": yA_neg,
                    "negabs_runB": yB_neg,
                }).to_csv(comb_csv, index=False)

                pos_title = (f"Bin {BIN_VALUE} — Mirror Positive Gradients "
                             f"[{tag}] (cos={cos_pos:.4f})")
                neg_title = (f"Bin {BIN_VALUE} — Mirror Negative Gradients |abs| "
                             f"[{tag}] (cos={cos_neg:.4f})")

                mirror_plot(
                    x_grid, yA_pos, yB_pos,
                    title=pos_title,
                    outfile=os.path.join(PLOTS_DIR, f"{tag}__mirror_pos.png")
                )
                mirror_plot(
                    x_grid, yA_neg, yB_neg,
                    title=neg_title,
                    outfile=os.path.join(PLOTS_DIR, f"{tag}__mirror_negabs.png")
                )

                # ---- NEW: Stats for volcano + ML agreement
                pvals, log2fc = ttest_and_log2fc(X, Y, pos_ids, neg_ids, eps=1e-12)

                # Save per-feature stats + raw grads + mean grad
                stats_csv = os.path.join(CSV_DIR, f"stats_and_grads__{tag}.csv")
                pd.DataFrame({
                    "feature": list(feature_cols),
                    "log2FC": log2fc,
                    "pval": pvals,
                    "neglog10p": -np.log10(np.clip(pvals, 1e-300, 1.0)),
                    "grad_runA_full": grad_A,
                    "grad_runB_full": grad_B,
                    "grad_mean": grad_mean,
                    "signed_stat": log2fc * (-np.log10(np.clip(pvals, 1e-300, 1.0))),
                }).to_csv(stats_csv, index=False)

                volcano_plot(
                    log2fc, pvals, LFC_LEFT, LFC_RIGHT, ALPHA,
                    title=f"Bin {BIN_VALUE} — Volcano ({tag})",
                    outfile=os.path.join(PLOTS_DIR, f"{tag}__volcano.png")
                )
                r = signed_agreement_plot(
                    log2fc, pvals, grad_mean,
                    LFC_LEFT, LFC_RIGHT, ALPHA,
                    title_prefix=f"Bin {BIN_VALUE} — Signed agreement ({tag})",
                    outfile=os.path.join(PLOTS_DIR, f"{tag}__signed_agreement.png")
                )

                # summary JSON directly under bin/
                with open(os.path.join(OUT_DIR, f"summary__{tag}.json"), "w") as fC:
                    json.dump({
                        "bin": BIN_VALUE,
                        "grouping": {"pos_ids": list(pos_ids), "neg_ids": list(neg_ids)},
                        "comparison": "Run A vs Run B",
                        "cosine_pos": cos_pos,
                        "cosine_neg_abs": cos_neg,
                        "pearson_signed_agreement": r,
                        "thresholds": {"alpha": ALPHA, "lfc_left": LFC_LEFT, "lfc_right": LFC_RIGHT},
                        "paths": {
                            "combined_csv_for_mirror": comb_csv,
                            "stats_and_grads_csv": stats_csv,
                            "plots_dir": PLOTS_DIR,
                        }
                    }, fC, indent=2)

                print(f"  [COMPARE] {tag} | Cos(pos)={cos_pos:.6f} Cos(neg|abs|)={cos_neg:.6f}  r={r:.3f}")

            # run all groupings -> write combined CSVs/plots/JSONs
            num_classes = int(np.max(Y)) + 1
            for (pos_ids, neg_ids) in groupings:
                for idx in (*pos_ids, *neg_ids):
                    assert 0 <= idx < num_classes, f"Class index {idx} out of range 0..{num_classes-1}"
                save_compare_only(pos_ids, neg_ids)

            # --- Split step: create split CSVs into csv/result/ ---
            split_outputs = process_folder(CSV_DIR, int(BIN_VALUE))

            # Move split files from csv/result/ -> csv/ and delete originals
            result_dir = os.path.join(CSV_DIR, "result")
            moved = []
            if os.path.isdir(result_dir):
                for fname in os.listdir(result_dir):
                    src = os.path.join(result_dir, fname)
                    dst = os.path.join(CSV_DIR, fname)
                    os.replace(src, dst)
                    moved.append(dst)
                try: os.rmdir(result_dir)
                except OSError: pass

            # Delete original combined CSVs (keep only split)
            for fname in os.listdir(CSV_DIR):
                if fname.lower().endswith(".csv") and fname.startswith("grads_AB__"):
                    try:
                        os.remove(os.path.join(CSV_DIR, fname))
                    except Exception as e:
                        print(f"[WARN] Could not remove {fname}: {e}")

            # Save manifest of final CSVs
            with open(os.path.join(OUT_DIR, "split_manifest.json"), "w") as fman:
                json.dump({
                    "bin": BIN_VALUE,
                    "final_csvs": sorted([os.path.basename(p) for p in moved]),
                }, fman, indent=2)

        finally:
            try:
                if models_A is not None:
                    for _m in models_A: del _m
                del models_A
            except: pass
            try:
                if models_B is not None:
                    for _m in models_B: del _m
                del models_B
            except: pass
            for v in ["X","Y","fdf"]:
                try: del globals()[v]
                except: pass
            hard_free()

    print("\nAll bins processed. (bin-root outputs; csv keeps only split files)\n")

# ---- run ----
if __name__ == "__main__":
    main()


Actual samples but the volcano graph has TIC normalization

In [ ]:
# -*- coding: utf-8 -*-
import os, re, json, gc, sys, ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optional (not used in plotting)
import seaborn as sns  # noqa: F401
from scipy.signal import find_peaks  # noqa: F401
from scipy.signal import savgol_filter  # noqa: F401
from scipy.ndimage import gaussian_filter1d  # noqa: F401
from scipy.linalg import svd  # noqa: F401
from scipy import stats  # <-- NEW: t-test

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers
from sklearn.model_selection import KFold

# ----------------------------
# Repro & GPU memory growth
# ----------------------------
np.random.seed(42)
tf.random.set_seed(42)
try:
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for _gpu in gpus:
        tf.config.experimental.set_memory_growth(_gpu, True)
except Exception:
    pass

def hard_free():
    """Aggressively release memory after each bin."""
    try: plt.close('all')
    except: pass
    try: tf.keras.backend.clear_session()
    except: pass
    try: gc.collect(); gc.collect()
    except: pass
    try:
        import ctypes, platform
        if platform.system().lower() == "linux":
            ctypes.CDLL("libc.so.6").malloc_trim(0)
    except: pass

# ----------------------------
# Grouped log-odds gradient
# ----------------------------
@tf.function(reduce_retracing=True)
def _group_logodds_grad_for_model(x1, model, pos_ids, neg_ids, eps):
    pos_ids = tf.constant(pos_ids, dtype=tf.int32)
    neg_ids = tf.constant(neg_ids, dtype=tf.int32)
    with tf.GradientTape() as tape:
        tape.watch(x1)
        p = model(x1, training=False)  # (1, C)
        p_pos = tf.reduce_sum(tf.gather(p, pos_ids, axis=1), axis=1)  # (1,)
        p_neg = tf.reduce_sum(tf.gather(p, neg_ids, axis=1), axis=1)  # (1,)
        log_odds = tf.math.log(p_pos + eps) - tf.math.log(p_neg + eps)
    g = tape.gradient(log_odds, x1)  # (1, D)
    return tf.squeeze(g, axis=0)     # (D,)

def compute_avg_group_logodds_gradient(X: np.ndarray, models: list, pos_ids=(2,3), neg_ids=(0,1), eps: float = 1e-8) -> np.ndarray:
    X_t = tf.convert_to_tensor(X, dtype=tf.float32)
    N = int(X_t.shape[0])
    sample_grads = []
    for i in range(N):
        x_i = X_t[i:i+1]
        grads_over_models = []
        for m in models:
            g = _group_logodds_grad_for_model(x_i, m, pos_ids, neg_ids, eps)
            grads_over_models.append(g)
        g_avg_models = tf.reduce_mean(tf.stack(grads_over_models, axis=0), axis=0)
        sample_grads.append(g_avg_models)
    avg_grad = tf.reduce_mean(tf.stack(sample_grads, axis=0), axis=0)
    return avg_grad.numpy()

# ----------------------------
# Model
# ----------------------------
def build_model(input_dim: int, num_classes: int):
    model = Sequential([
        Dense(128, input_dim=input_dim, activation='relu', kernel_regularizer=regularizers.l1(0.01)),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])
    return model

# ----------------------------
# Helpers (cosine + plotting)
# ----------------------------
def cosine_sim(a: np.ndarray, b: np.ndarray, eps: float = 1e-12) -> float:
    a = np.asarray(a, dtype=float).ravel()
    b = np.asarray(b, dtype=float).ravel()
    n = min(a.size, b.size)
    a = a[:n]; b = b[:n]
    denom = (np.linalg.norm(a) * np.linalg.norm(b)) + eps
    return float(np.dot(a, b) / denom)

def mirror_plot(x, top_y, bottom_y, title, outfile):
    plt.figure(figsize=(10, 5))
    plt.plot(x, top_y, linewidth=1.0, label="Run A")
    plt.plot(x, -bottom_y, linewidth=1.0, label="Run B (mirrored)")
    plt.axhline(0.0, linewidth=0.8)
    plt.xlabel("m/z (approx grid)")
    plt.ylabel("Gradient magnitude")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.savefig(outfile, dpi=200)
    plt.close()

# ----------------------------
# NEW: Stats & plotting helpers for volcano + signed agreement
# ----------------------------
def ttest_and_log2fc(X: np.ndarray, y: np.ndarray, pos_ids, neg_ids, eps: float = 1e-12):
    """Welch's t-test (pos vs neg) per feature + log2FC (pos/neg)."""
    pos_mask = np.isin(y, list(pos_ids))
    neg_mask = np.isin(y, list(neg_ids))
    Xp = X[pos_mask]; Xn = X[neg_mask]
    # Handle edge cases gracefully
    if Xp.shape[0] < 2 or Xn.shape[0] < 2:
        # fall back to NaNs so plots show empty
        pvals = np.full(X.shape[1], np.nan)
        log2fc = np.full(X.shape[1], np.nan)
        return pvals, log2fc

    # Welch's t-test, axis=0 over features
    tstat, pvals = stats.ttest_ind(Xp, Xn, axis=0, equal_var=False, nan_policy='omit')
    mu_p = np.nanmean(Xp, axis=0)
    mu_n = np.nanmean(Xn, axis=0)
    log2fc = np.log2((mu_p + eps) / (mu_n + eps))
    # Replace any nan/inf from degenerate features
    pvals = np.where(np.isfinite(pvals), pvals, 1.0)
    log2fc = np.where(np.isfinite(log2fc), log2fc, 0.0)
    return pvals, log2fc

def volcano_plot(log2fc: np.ndarray, pvals: np.ndarray,
                 lfc_left: float, lfc_right: float, alpha: float,
                 title: str, outfile: str):
    xl = log2fc
    yl = -np.log10(np.clip(pvals, 1e-300, 1.0))
    sig_up   = (xl >= lfc_right) & (pvals <= alpha)
    sig_down = (xl <= lfc_left)  & (pvals <= alpha)
    other    = ~(sig_up | sig_down)

    plt.figure(figsize=(10,6))
    plt.scatter(xl[other], yl[other], s=15, alpha=0.6, color='lightgray')
    plt.scatter(xl[sig_down], yl[sig_down], s=30, alpha=0.9, color='C0')  # blue
    plt.scatter(xl[sig_up], yl[sig_up], s=30, alpha=0.9, color='C3')      # red
    # thresholds
    plt.axvline(lfc_left,  ls='--', lw=1, color='k')
    plt.axvline(lfc_right, ls='--', lw=1, color='k')
    plt.axhline(-np.log10(alpha), ls='--', lw=1, color='k')
    plt.xlabel("log2 fold-change (pos / neg)")
    plt.ylabel(r"$-\log_{10}(p)$")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(outfile, dpi=200)
    plt.close()

def signed_agreement_plot(log2fc: np.ndarray, pvals: np.ndarray, grad_mean: np.ndarray,
                          lfc_left: float, lfc_right: float, alpha: float,
                          title_prefix: str, outfile: str):
    signed_stat = log2fc * (-np.log10(np.clip(pvals, 1e-300, 1.0)))
    # Pearson r (ignore NaNs)
    mask = np.isfinite(signed_stat) & np.isfinite(grad_mean)
    if np.sum(mask) >= 2:
        r = np.corrcoef(signed_stat[mask], grad_mean[mask])[0,1]
    else:
        r = np.nan

    sig_up   = (log2fc >= lfc_right) & (pvals <= alpha)
    sig_down = (log2fc <= lfc_left)  & (pvals <= alpha)
    other    = ~(sig_up | sig_down)

    plt.figure(figsize=(10,6))
    plt.scatter(signed_stat[other], grad_mean[other], s=15, alpha=0.6, color='lightgray')
    plt.scatter(signed_stat[sig_down], grad_mean[sig_down], s=30, alpha=0.9, color='C0')
    plt.scatter(signed_stat[sig_up],   grad_mean[sig_up],   s=30, alpha=0.9, color='C3')
    plt.axvline(0.0, ls='--', lw=1, color='k'); plt.axhline(0.0, ls='--', lw=1, color='k')
    plt.xlabel(r"log2FC × $-\log_{10}(p)$  (signed)")
    plt.ylabel("Average log-odds gradient (RunA/B mean)")
    plt.title(f"{title_prefix}  (Pearson r = {r:.3f})")
    plt.tight_layout()
    plt.savefig(outfile, dpi=200)
    plt.close()
    return float(r)

# ----------------------------
# Splitter (4 CSVs per combined CSV) with bin prefix
# ----------------------------
TARGET_COLS = ["pos_runA", "pos_runB", "negabs_runA", "negabs_runB"]
MZ_COL = "m/z"

def split_csv(input_path: str, out_dir: str, bin_value: int) -> list[str]:
    """Split one combined CSV into 4 CSVs, prefixing filenames with bin number."""
    df = pd.read_csv(input_path)
    if MZ_COL not in df.columns:
        print(f"[SKIP] {input_path} (no '{MZ_COL}' column)")
        return []
    available_targets = [c for c in TARGET_COLS if c in df.columns]
    if not available_targets:
        print(f"[SKIP] {input_path} (none of {TARGET_COLS} found)")
        return []
    base = os.path.splitext(os.path.basename(input_path))[0]
    written = []
    for col in available_targets:
        out_df = df[[MZ_COL, col]].copy()
        out_path = os.path.join(out_dir, f"bin{bin_value}_{base}_{col}.csv")
        out_df.to_csv(out_path, index=False)
        written.append(out_path)
    return written

def process_folder(folder_path: str, bin_value: int) -> list[str]:
    """Split all CSVs in a folder to 'result/' and return list of result paths."""
    if not os.path.isdir(folder_path):
        print(f"[WARN] Not a folder: {folder_path}")
        return []
    out_dir = os.path.join(folder_path, "result")
    os.makedirs(out_dir, exist_ok=True)

    all_outputs = []
    for fname in os.listdir(folder_path):
        if fname.lower().endswith(".csv"):
            fpath = os.path.join(folder_path, fname)
            print(f"Splitting {fpath} ...")
            outputs = split_csv(fpath, out_dir, bin_value)
            all_outputs.extend(outputs)

    print(f"Split done. Wrote {len(all_outputs)} files to {out_dir}")
    return all_outputs

# ----------------------------
# Config (edit these)
# ----------------------------
CSV_PATH    = r"F:/20251010/dataset_rt_batch.csv"   # input dataset with 'bin' and 'target'
EPOCHS      = 50
BATCH_SIZE  = 32
K_SPLITS    = 5
N_REPEATS   = 5
SEED_BASES  = [111, 777]                   # two independent runs
OUT_ROOT    = r"F:/20251018d/"              # everything goes directly under bin_<N>/
BIN_WHITELIST = None  # e.g. [35, 75]

# Volcano thresholds (edit to taste)
ALPHA = 0.05
LFC_LEFT, LFC_RIGHT = -1.0, 1.0

# >>> NEW: TIC normalization scale (used ONLY for volcano stats) <<<
TIC_SCALE = 1e10

# Fixed grouping list (no prompt, no AUTO)
GROUPINGS = [((2,),(1,)), ((2,),(0,)), ((1,),(0,)), ((3,),(2,)), ((3,),(0,1,2)), ((2,1),(0,)), ((3,),(1,)),  ((3,),(0,))]

# ----------------------------
# KFold+Repeats trainer
# ----------------------------
def train_kfold_repeats(X: np.ndarray, Y: np.ndarray, seed_base: int):
    kf = KFold(n_splits=K_SPLITS, shuffle=True, random_state=42)
    all_models = []
    num_classes = int(np.max(Y)) + 1
    for fold, (tr, va) in enumerate(kf.split(X, Y), 1):
        X_tr, y_tr = X[tr], Y[tr]
        X_va, y_va = X[va], Y[va]
        for r in range(N_REPEATS):
            seed = seed_base * 1000 + fold * 100 + r
            tf.keras.utils.set_random_seed(seed)
            np.random.seed(seed)
            m = build_model(X.shape[1], num_classes)
            m.fit(X_tr, y_tr, epochs=EPOCHS, batch_size=BATCH_SIZE,
                  validation_data=(X_va, y_va), verbose=0)
            all_models.append(m)
        print(f"[Seed base {seed_base}] Fold {fold}/{K_SPLITS} trained {N_REPEATS} models (total: {len(all_models)})")
    return all_models

# ----------------------------
# MAIN (bin-root outputs + keep only split CSVs)
# ----------------------------
def main():
    df = pd.read_csv(CSV_PATH)
    # Discover bins
    bins_found = sorted([b for b in df["bin"].dropna().unique().tolist()])
    if BIN_WHITELIST is not None:
        bins_to_process = [b for b in bins_found if b in set(BIN_WHITELIST)]
    else:
        bins_to_process = bins_found
    if len(bins_to_process) == 0:
        raise ValueError(f"No 'bin' values found in {CSV_PATH}")

    # Keep only classes 0..3 by default
    df = df[df["target"].astype(int).isin([0,1,2,3])].copy()
    unique_labels = np.sort(df["target"].astype(int).unique())
    assert unique_labels[0] == 0 and np.array_equal(unique_labels, np.arange(unique_labels[-1] + 1)), \
        f"Non-contiguous labels detected: {unique_labels}. Please remap to 0..C-1."
    groupings = GROUPINGS
    print(f"\nUsing fixed grouping(s): {groupings}")

    os.makedirs(OUT_ROOT, exist_ok=True)

    for BIN_VALUE in bins_to_process:
        print(f"\n================= BIN {BIN_VALUE} =================")
        models_A = models_B = None
        X = Y = fdf = None

        try:
            fdf = df[df["bin"] == BIN_VALUE].copy()
            if fdf.empty:
                print(f"[WARN] No rows for bin {BIN_VALUE}; skipping.")
                continue

            # Normalize all features except ['bin','target'] within this bin
            cols_to_norm = fdf.columns.difference(['bin', 'target'])
            fdf[cols_to_norm] = fdf[cols_to_norm].apply(lambda x: x / (x.max() + 1.0))

            Y = fdf["target"].astype(int).to_numpy()
            feature_cols = fdf.columns.difference(['bin', 'target'])
            X = np.nan_to_num(fdf[feature_cols].to_numpy(), copy=False).astype(np.float32)

            if X.shape[0] < 2 or X.shape[1] < 1:
                print(f"[WARN] Insufficient data for bin {BIN_VALUE} (samples={X.shape[0]}, dim={X.shape[1]}). Skipping.")
                continue

            print(f"Bin {BIN_VALUE}: samples={X.shape[0]}, dim={X.shape[1]}  class_counts="
                  f"{dict(zip(*np.unique(Y, return_counts=True)))}")

            # Output dirs per bin (everything under bin folder)
            OUT_DIR   = os.path.join(OUT_ROOT, f"bin_{str(BIN_VALUE).replace('.', '_')}")
            CSV_DIR   = os.path.join(OUT_DIR, "csv")
            PLOTS_DIR = os.path.join(OUT_DIR, "plots")
            os.makedirs(OUT_DIR, exist_ok=True)
            os.makedirs(CSV_DIR, exist_ok=True)
            os.makedirs(PLOTS_DIR, exist_ok=True)

            # Train ensembles ONCE per run (A, B)
            models_A = train_kfold_repeats(X, Y, seed_base=SEED_BASES[0])
            models_B = train_kfold_repeats(X, Y, seed_base=SEED_BASES[1])

            # grid helper (for your mirror plots)
            def _make_grid(n):
                n_grid = min(10000, n)
                x_grid = np.arange(600, 600 + 0.1 * n_grid, 0.1)[:n_grid]
                return n_grid, x_grid

            def save_compare_only(pos_ids, neg_ids):
                # ---- Gradients (Run A & B) — unchanged, NO TIC normalization
                grad_A = compute_avg_group_logodds_gradient(X, models_A, pos_ids=pos_ids, neg_ids=neg_ids, eps=1e-8)
                grad_B = compute_avg_group_logodds_gradient(X, models_B, pos_ids=pos_ids, neg_ids=neg_ids, eps=1e-8)
                grad_mean = (grad_A + grad_B) / 2.0

                pos_tag = "_".join(map(str, pos_ids))
                neg_tag = "_".join(map(str, neg_ids))
                tag = f"pos_{pos_tag}__neg_{neg_tag}"

                # ---- Mirror plots (unchanged from your flow)
                n_grid, x_grid = _make_grid(min(grad_A.size, grad_B.size))
                yA = grad_A[:n_grid]; yB = grad_B[:n_grid]
                yA_pos = np.where(yA > 0, yA, 0.0)
                yB_pos = np.where(yB > 0, yB, 0.0)
                yA_neg = np.where(yA < 0, -yA, 0.0)  # abs
                yB_neg = np.where(yB < 0, -yB, 0.0)

                cos_pos = cosine_sim(yA_pos, yB_pos)
                cos_neg = cosine_sim(yA_neg, yB_neg)

                comb_csv = os.path.join(CSV_DIR, f"grads_AB__{tag}.csv")
                pd.DataFrame({
                    "m/z": x_grid,
                    "grad_runA": yA,
                    "grad_runB": yB,
                    "pos_runA": yA_pos,
                    "pos_runB": yB_pos,
                    "negabs_runA": yA_neg,
                    "negabs_runB": yB_neg,
                }).to_csv(comb_csv, index=False)

                pos_title = (f"Bin {BIN_VALUE} — Mirror Positive Gradients "
                             f"[{tag}] (cos={cos_pos:.4f})")
                neg_title = (f"Bin {BIN_VALUE} — Mirror Negative Gradients |abs| "
                             f"[{tag}] (cos={cos_neg:.4f})")

                mirror_plot(
                    x_grid, yA_pos, yB_pos,
                    title=pos_title,
                    outfile=os.path.join(PLOTS_DIR, f"{tag}__mirror_pos.png")
                )
                mirror_plot(
                    x_grid, yA_neg, yB_neg,
                    title=neg_title,
                    outfile=os.path.join(PLOTS_DIR, f"{tag}__mirror_negabs.png")
                )

                # ---- NEW: TIC normalization ONLY for volcano statistics
                # Compute per-sample TIC on X, then scale by TIC_SCALE
                tic = np.sum(X, axis=1)
                tic = np.clip(tic, 1e-12, None)
                X_tic = X / (tic[:, None] / TIC_SCALE)

                # ---- Stats for volcano + ML agreement (now using TIC-normalized data)
                pvals, log2fc = ttest_and_log2fc(X_tic, Y, pos_ids, neg_ids, eps=1e-12)

                # Save per-feature stats + raw grads + mean grad
                stats_csv = os.path.join(CSV_DIR, f"stats_and_grads__{tag}.csv")
                pd.DataFrame({
                    "feature": list(feature_cols),
                    "log2FC": log2fc,
                    "pval": pvals,
                    "neglog10p": -np.log10(np.clip(pvals, 1e-300, 1.0)),
                    "grad_runA_full": grad_A,
                    "grad_runB_full": grad_B,
                    "grad_mean": grad_mean,
                    "signed_stat": log2fc * (-np.log10(np.clip(pvals, 1e-300, 1.0))),
                }).to_csv(stats_csv, index=False)

                volcano_plot(
                    log2fc, pvals, LFC_LEFT, LFC_RIGHT, ALPHA,
                    title=f"Bin {BIN_VALUE} — Volcano ({tag})",
                    outfile=os.path.join(PLOTS_DIR, f"{tag}__volcano.png")
                )
                r = signed_agreement_plot(
                    log2fc, pvals, grad_mean,
                    LFC_LEFT, LFC_RIGHT, ALPHA,
                    title_prefix=f"Bin {BIN_VALUE} — Signed agreement ({tag})",
                    outfile=os.path.join(PLOTS_DIR, f"{tag}__signed_agreement.png")
                )

                # summary JSON directly under bin/
                with open(os.path.join(OUT_DIR, f"summary__{tag}.json"), "w") as fC:
                    json.dump({
                        "bin": BIN_VALUE,
                        "grouping": {"pos_ids": list(pos_ids), "neg_ids": list(neg_ids)},
                        "comparison": "Run A vs Run B",
                        "cosine_pos": cos_pos,
                        "cosine_neg_abs": cos_neg,
                        "pearson_signed_agreement": r,
                        "thresholds": {"alpha": ALPHA, "lfc_left": LFC_LEFT, "lfc_right": LFC_RIGHT},
                        "paths": {
                            "combined_csv_for_mirror": comb_csv,
                            "stats_and_grads_csv": stats_csv,
                            "plots_dir": PLOTS_DIR,
                        }
                    }, fC, indent=2)

                print(f"  [COMPARE] {tag} | Cos(pos)={cos_pos:.6f} Cos(neg|abs|)={cos_neg:.6f}  r={r:.3f}")

            # run all groupings -> write combined CSVs/plots/JSONs
            num_classes = int(np.max(Y)) + 1
            for (pos_ids, neg_ids) in groupings:
                for idx in (*pos_ids, *neg_ids):
                    assert 0 <= idx < num_classes, f"Class index {idx} out of range 0..{num_classes-1}"
                save_compare_only(pos_ids, neg_ids)

            # --- Split step: create split CSVs into csv/result/ ---
            split_outputs = process_folder(CSV_DIR, int(BIN_VALUE))

            # Move split files from csv/result/ -> csv/ and delete originals
            result_dir = os.path.join(CSV_DIR, "result")
            moved = []
            if os.path.isdir(result_dir):
                for fname in os.listdir(result_dir):
                    src = os.path.join(result_dir, fname)
                    dst = os.path.join(CSV_DIR, fname)
                    os.replace(src, dst)
                    moved.append(dst)
                try: os.rmdir(result_dir)
                except OSError: pass

            # Delete original combined CSVs (keep only split)
            for fname in os.listdir(CSV_DIR):
                if fname.lower().endswith(".csv") and fname.startswith("grads_AB__"):
                    try:
                        os.remove(os.path.join(CSV_DIR, fname))
                    except Exception as e:
                        print(f"[WARN] Could not remove {fname}: {e}")

            # Save manifest of final CSVs
            with open(os.path.join(OUT_DIR, "split_manifest.json"), "w") as fman:
                json.dump({
                    "bin": BIN_VALUE,
                    "final_csvs": sorted([os.path.basename(p) for p in moved]),
                }, fman, indent=2)

        finally:
            try:
                if models_A is not None:
                    for _m in models_A: del _m
                del models_A
            except: pass
            try:
                if models_B is not None:
                    for _m in models_B: del _m
                del models_B
            except: pass
            for v in ["X","Y","fdf"]:
                try: del globals()[v]
                except: pass
            hard_free()

    print("\nAll bins processed. (bin-root outputs; csv keeps only split files)\n")

# ---- run ----
if __name__ == "__main__":
    main()



Using fixed grouping(s): [((2,), (1,)), ((2,), (0,)), ((1,), (0,)), ((3,), (2,)), ((3,), (0, 1, 2)), ((2, 1), (0,)), ((3,), (1,)), ((3,), (0,))]

================= BIN 5 =================
Bin 5: samples=118, dim=13692  class_counts={0: 33, 1: 30, 2: 26, 3: 29}


c:\Users\benja\miniconda3\envs\vae\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[Seed base 111] Fold 1/5 trained 5 models (total: 5)
[Seed base 111] Fold 2/5 trained 5 models (total: 10)
[Seed base 111] Fold 3/5 trained 5 models (total: 15)
[Seed base 111] Fold 4/5 trained 5 models (total: 20)
[Seed base 111] Fold 5/5 trained 5 models (total: 25)
[Seed base 777] Fold 1/5 trained 5 models (total: 5)
[Seed base 777] Fold 2/5 trained 5 models (total: 10)
[Seed base 777] Fold 3/5 trained 5 models (total: 15)
[Seed base 777] Fold 4/5 trained 5 models (total: 20)
[Seed base 777] Fold 5/5 trained 5 models (total: 25)
  [COMPARE] pos_2__neg_1 | Cos(pos)=0.987876 Cos(neg|abs|)=0.986822  r=0.547
  [COMPARE] pos_2__neg_0 | Cos(pos)=0.986543 Cos(neg|abs|)=0.985502  r=0.519
  [COMPARE] pos_1__neg_0 | Cos(pos)=0.888078 Cos(neg|abs|)=0.928564  r=0.416
  [COMPARE] pos_3__neg_2 | Cos(pos)=0.950173 Cos(neg|abs|)=0.972997  r=0.491
  [COMPARE] pos_3__neg_0_1_2 | Cos(pos)=0.969280 Cos(neg|abs|)=0.980436  r=0.430
  [COMPARE] pos_2_1__neg_0 | Cos(pos)=0.977046 Cos(neg|abs|)=0.979960  r=

c:\Users\benja\miniconda3\envs\vae\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[Seed base 111] Fold 1/5 trained 5 models (total: 5)
[Seed base 111] Fold 2/5 trained 5 models (total: 10)
[Seed base 111] Fold 3/5 trained 5 models (total: 15)
[Seed base 111] Fold 4/5 trained 5 models (total: 20)
[Seed base 111] Fold 5/5 trained 5 models (total: 25)
[Seed base 777] Fold 1/5 trained 5 models (total: 5)
[Seed base 777] Fold 2/5 trained 5 models (total: 10)
[Seed base 777] Fold 3/5 trained 5 models (total: 15)
[Seed base 777] Fold 4/5 trained 5 models (total: 20)
[Seed base 777] Fold 5/5 trained 5 models (total: 25)
  [COMPARE] pos_2__neg_1 | Cos(pos)=0.989463 Cos(neg|abs|)=0.979476  r=0.599
  [COMPARE] pos_2__neg_0 | Cos(pos)=0.985870 Cos(neg|abs|)=0.980388  r=0.549
  [COMPARE] pos_1__neg_0 | Cos(pos)=0.914871 Cos(neg|abs|)=0.921428  r=0.421
  [COMPARE] pos_3__neg_2 | Cos(pos)=0.983107 Cos(neg|abs|)=0.973670  r=0.479
  [COMPARE] pos_3__neg_0_1_2 | Cos(pos)=0.992625 Cos(neg|abs|)=0.973871  r=0.401
  [COMPARE] pos_2_1__neg_0 | Cos(pos)=0.965530 Cos(neg|abs|)=0.971910  r=

...

[Seed base 111] Fold 1/5 trained 5 models (total: 5)
[Seed base 111] Fold 2/5 trained 5 models (total: 10)
[Seed base 111] Fold 3/5 trained 5 models (total: 15)
[Seed base 111] Fold 4/5 trained 5 models (total: 20)
[Seed base 111] Fold 5/5 trained 5 models (total: 25)
[Seed base 777] Fold 1/5 trained 5 models (total: 5)
[Seed base 777] Fold 2/5 trained 5 models (total: 10)
[Seed base 777] Fold 3/5 trained 5 models (total: 15)
[Seed base 777] Fold 4/5 trained 5 models (total: 20)
[Seed base 777] Fold 5/5 trained 5 models (total: 25)
  [COMPARE] pos_2__neg_1 | Cos(pos)=0.730192 Cos(neg|abs|)=0.852749  r=0.623
  [COMPARE] pos_2__neg_0 | Cos(pos)=0.933359 Cos(neg|abs|)=0.889741  r=0.353
  [COMPARE] pos_1__neg_0 | Cos(pos)=0.926483 Cos(neg|abs|)=0.842590  r=0.395
  [COMPARE] pos_3__neg_2 | Cos(pos)=0.859105 Cos(neg|abs|)=0.973880  r=0.239
  [COMPARE] pos_3__neg_0_1_2 | Cos(pos)=0.860420 Cos(neg|abs|)=0.974485  r=0.159
  [COMPARE] pos_2_1__neg_0 | Cos(pos)=0.943897 Cos(neg|abs|)=0.892494  r=

c:\Users\benja\miniconda3\envs\vae\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[Seed base 111] Fold 1/5 trained 5 models (total: 5)
[Seed base 111] Fold 2/5 trained 5 models (total: 10)
[Seed base 111] Fold 3/5 trained 5 models (total: 15)
[Seed base 111] Fold 4/5 trained 5 models (total: 20)
[Seed base 111] Fold 5/5 trained 5 models (total: 25)
[Seed base 777] Fold 1/5 trained 5 models (total: 5)
[Seed base 777] Fold 2/5 trained 5 models (total: 10)
[Seed base 777] Fold 3/5 trained 5 models (total: 15)
[Seed base 777] Fold 4/5 trained 5 models (total: 20)
[Seed base 777] Fold 5/5 trained 5 models (total: 25)
  [COMPARE] pos_2__neg_1 | Cos(pos)=0.633620 Cos(neg|abs|)=0.643979  r=0.628
  [COMPARE] pos_2__neg_0 | Cos(pos)=0.947583 Cos(neg|abs|)=0.715467  r=0.411
  [COMPARE] pos_1__neg_0 | Cos(pos)=0.946522 Cos(neg|abs|)=0.646888  r=0.427
  [COMPARE] pos_3__neg_2 | Cos(pos)=0.636357 Cos(neg|abs|)=0.965171  r=0.161
  [COMPARE] pos_3__neg_0_1_2 | Cos(pos)=0.761158 Cos(neg|abs|)=0.965264  r=0.072
  [COMPARE] pos_2_1__neg_0 | Cos(pos)=0.957448 Cos(neg|abs|)=0.703140  r=

c:\Users\benja\miniconda3\envs\vae\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[Seed base 111] Fold 1/5 trained 5 models (total: 5)
[Seed base 111] Fold 2/5 trained 5 models (total: 10)
[Seed base 111] Fold 3/5 trained 5 models (total: 15)
[Seed base 111] Fold 4/5 trained 5 models (total: 20)
[Seed base 111] Fold 5/5 trained 5 models (total: 25)
[Seed base 777] Fold 1/5 trained 5 models (total: 5)
[Seed base 777] Fold 2/5 trained 5 models (total: 10)
[Seed base 777] Fold 3/5 trained 5 models (total: 15)
[Seed base 777] Fold 4/5 trained 5 models (total: 20)
[Seed base 777] Fold 5/5 trained 5 models (total: 25)


With synthetic dataset of the same dimension

In [3]:
# -*- coding: utf-8 -*-
import os, re, json, gc, sys, ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optional (not used in plotting)
import seaborn as sns  # noqa: F401
from scipy.signal import find_peaks  # noqa: F401
from scipy.signal import savgol_filter  # noqa: F401
from scipy.ndimage import gaussian_filter1d  # noqa: F401
from scipy.linalg import svd  # noqa: F401
from scipy import stats  # <-- NEW: t-test

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers
from sklearn.model_selection import KFold

# ----------------------------
# Repro & GPU memory growth
# ----------------------------
np.random.seed(42)
tf.random.set_seed(42)
try:
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for _gpu in gpus:
        tf.config.experimental.set_memory_growth(_gpu, True)
except Exception:
    pass

def hard_free():
    """Aggressively release memory after each bin."""
    try: plt.close('all')
    except: pass
    try: tf.keras.backend.clear_session()
    except: pass
    try: gc.collect(); gc.collect()
    except: pass
    try:
        import ctypes, platform
        if platform.system().lower() == "linux":
            ctypes.CDLL("libc.so.6").malloc_trim(0)
    except: pass

# ----------------------------
# Grouped log-odds gradient
# ----------------------------
@tf.function(reduce_retracing=True)
def _group_logodds_grad_for_model(x1, model, pos_ids, neg_ids, eps):
    pos_ids = tf.constant(pos_ids, dtype=tf.int32)
    neg_ids = tf.constant(neg_ids, dtype=tf.int32)
    with tf.GradientTape() as tape:
        tape.watch(x1)
        p = model(x1, training=False)  # (1, C)
        p_pos = tf.reduce_sum(tf.gather(p, pos_ids, axis=1), axis=1)  # (1,)
        p_neg = tf.reduce_sum(tf.gather(p, neg_ids, axis=1), axis=1)  # (1,)
        log_odds = tf.math.log(p_pos + eps) - tf.math.log(p_neg + eps)
    g = tape.gradient(log_odds, x1)  # (1, D)
    return tf.squeeze(g, axis=0)     # (D,)

def compute_avg_group_logodds_gradient(X: np.ndarray, models: list, pos_ids=(2,3), neg_ids=(0,1), eps: float = 1e-8) -> np.ndarray:
    X_t = tf.convert_to_tensor(X, dtype=tf.float32)
    N = int(X_t.shape[0])
    sample_grads = []
    for i in range(N):
        x_i = X_t[i:i+1]
        grads_over_models = []
        for m in models:
            g = _group_logodds_grad_for_model(x_i, m, pos_ids, neg_ids, eps)
            grads_over_models.append(g)
        g_avg_models = tf.reduce_mean(tf.stack(grads_over_models, axis=0), axis=0)
        sample_grads.append(g_avg_models)
    avg_grad = tf.reduce_mean(tf.stack(sample_grads, axis=0), axis=0)
    return avg_grad.numpy()

# ----------------------------
# Model
# ----------------------------
def build_model(input_dim: int, num_classes: int):
    model = Sequential([
        Dense(128, input_dim=input_dim, activation='relu', kernel_regularizer=regularizers.l1(0.01)),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])
    return model

# ----------------------------
# Helpers (cosine + plotting)
# ----------------------------
def cosine_sim(a: np.ndarray, b: np.ndarray, eps: float = 1e-12) -> float:
    a = np.asarray(a, dtype=float).ravel()
    b = np.asarray(b, dtype=float).ravel()
    n = min(a.size, b.size)
    a = a[:n]; b = b[:n]
    denom = (np.linalg.norm(a) * np.linalg.norm(b)) + eps
    return float(np.dot(a, b) / denom)

def mirror_plot(x, top_y, bottom_y, title, outfile):
    plt.figure(figsize=(10, 5))
    plt.plot(x, top_y, linewidth=1.0, label="Run A")
    plt.plot(x, -bottom_y, linewidth=1.0, label="Run B (mirrored)")
    plt.axhline(0.0, linewidth=0.8)
    plt.xlabel("m/z (approx grid)")
    plt.ylabel("Gradient magnitude")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.savefig(outfile, dpi=200)
    plt.close()

# ----------------------------
# Stats & plotting helpers for volcano + signed agreement
# ----------------------------
def ttest_and_log2fc(X: np.ndarray, y: np.ndarray, pos_ids, neg_ids, eps: float = 1e-12):
    """Welch's t-test (pos vs neg) per feature + log2FC (pos/neg)."""
    pos_mask = np.isin(y, list(pos_ids))
    neg_mask = np.isin(y, list(neg_ids))
    Xp = X[pos_mask]; Xn = X[neg_mask]
    # Handle edge cases gracefully
    if Xp.shape[0] < 2 or Xn.shape[0] < 2:
        pvals = np.full(X.shape[1], np.nan)
        log2fc = np.full(X.shape[1], np.nan)
        return pvals, log2fc

    # Welch's t-test, axis=0 over features
    _tstat, pvals = stats.ttest_ind(Xp, Xn, axis=0, equal_var=False, nan_policy='omit')
    mu_p = np.nanmean(Xp, axis=0)
    mu_n = np.nanmean(Xn, axis=0)
    log2fc = np.log2((mu_p + eps) / (mu_n + eps))
    # Replace any nan/inf from degenerate features
    pvals = np.where(np.isfinite(pvals), pvals, 1.0)
    log2fc = np.where(np.isfinite(log2fc), log2fc, 0.0)
    return pvals, log2fc

def volcano_plot(log2fc: np.ndarray, pvals: np.ndarray,
                 lfc_left: float, lfc_right: float, alpha: float,
                 title: str, outfile: str):
    xl = log2fc
    yl = -np.log10(np.clip(pvals, 1e-300, 1.0))
    sig_up   = (xl >= lfc_right) & (pvals <= alpha)
    sig_down = (xl <= lfc_left)  & (pvals <= alpha)
    other    = ~(sig_up | sig_down)

    plt.figure(figsize=(10,6))
    plt.scatter(xl[other], yl[other], s=15, alpha=0.6, color='lightgray')
    plt.scatter(xl[sig_down], yl[sig_down], s=30, alpha=0.9, color='C0')  # blue
    plt.scatter(xl[sig_up], yl[sig_up], s=30, alpha=0.9, color='C3')      # red
    # thresholds
    plt.axvline(lfc_left,  ls='--', lw=1, color='k')
    plt.axvline(lfc_right, ls='--', lw=1, color='k')
    plt.axhline(-np.log10(alpha), ls='--', lw=1, color='k')
    plt.xlabel("log2 fold-change (pos / neg)")
    plt.ylabel(r"$-\log_{10}(p)$")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(outfile, dpi=200)
    plt.close()

def signed_agreement_plot(log2fc: np.ndarray, pvals: np.ndarray, grad_mean: np.ndarray,
                          lfc_left: float, lfc_right: float, alpha: float,
                          title_prefix: str, outfile: str):
    signed_stat = log2fc * (-np.log10(np.clip(pvals, 1e-300, 1.0)))
    # Pearson r (ignore NaNs)
    mask = np.isfinite(signed_stat) & np.isfinite(grad_mean)
    if np.sum(mask) >= 2:
        r = np.corrcoef(signed_stat[mask], grad_mean[mask])[0,1]
    else:
        r = np.nan

    sig_up   = (log2fc >= lfc_right) & (pvals <= alpha)
    sig_down = (log2fc <= lfc_left)  & (pvals <= alpha)
    other    = ~(sig_up | sig_down)

    plt.figure(figsize=(10,6))
    plt.scatter(signed_stat[other], grad_mean[other], s=15, alpha=0.6, color='lightgray')
    plt.scatter(signed_stat[sig_down], grad_mean[sig_down], s=30, alpha=0.9, color='C0')
    plt.scatter(signed_stat[sig_up],   grad_mean[sig_up],   s=30, alpha=0.9, color='C3')
    plt.axvline(0.0, ls='--', lw=1, color='k'); plt.axhline(0.0, ls='--', lw=1, color='k')
    plt.xlabel(r"log2FC × $-\log_{10}(p)$  (signed)")
    plt.ylabel("Average log-odds gradient (RunA/B mean)")
    plt.title(f"{title_prefix}  (Pearson r = {r:.3f})")
    plt.tight_layout()
    plt.savefig(outfile, dpi=200)
    plt.close()
    return float(r)

# ----------------------------
# Splitter (4 CSVs per combined CSV) with bin prefix
# ----------------------------
TARGET_COLS = ["pos_runA", "pos_runB", "negabs_runA", "negabs_runB"]
MZ_COL = "m/z"

def split_csv(input_path: str, out_dir: str, bin_value: int) -> list[str]:
    """Split one combined CSV into 4 CSVs, prefixing filenames with bin number."""
    df = pd.read_csv(input_path)
    if MZ_COL not in df.columns:
        print(f"[SKIP] {input_path} (no '{MZ_COL}' column)")
        return []
    available_targets = [c for c in TARGET_COLS if c in df.columns]
    if not available_targets:
        print(f"[SKIP] {input_path} (none of {TARGET_COLS} found)")
        return []
    base = os.path.splitext(os.path.basename(input_path))[0]
    written = []
    for col in available_targets:
        out_df = df[[MZ_COL, col]].copy()
        out_path = os.path.join(out_dir, f"bin{bin_value}_{base}_{col}.csv")
        out_df.to_csv(out_path, index=False)
        written.append(out_path)
    return written

def process_folder(folder_path: str, bin_value: int) -> list[str]:
    """Split all CSVs in a folder to 'result/' and return list of result paths."""
    if not os.path.isdir(folder_path):
        print(f"[WARN] Not a folder: {folder_path}")
        return []
    out_dir = os.path.join(folder_path, "result")
    os.makedirs(out_dir, exist_ok=True)

    all_outputs = []
    for fname in os.listdir(folder_path):
        if fname.lower().endswith(".csv"):
            fpath = os.path.join(folder_path, fname)
            print(f"Splitting {fpath} ...")
            outputs = split_csv(fpath, out_dir, bin_value)
            all_outputs.extend(outputs)

    print(f"Split done. Wrote {len(all_outputs)} files to {out_dir}")
    return all_outputs

# ----------------------------
# Config (edit these)
# ----------------------------
CSV_PATH    = r"F:/20251010/dataset_rt_batch.csv"   # input dataset with 'bin' and 'target'
EPOCHS      = 50
BATCH_SIZE  = 32
K_SPLITS    = 5
N_REPEATS   = 5
SEED_BASES  = [111, 777]                   # two independent runs
OUT_ROOT    = r"F:/20251018e/"             # everything goes directly under bin_<N>/
BIN_WHITELIST = None  # e.g. [35, 75]

# Volcano thresholds
ALPHA = 0.05
LFC_LEFT, LFC_RIGHT = -0.5, 0.5

# Present but NOT used for volcano stats anymore
TIC_SCALE = 1e10

# Fixed grouping list (no prompt, no AUTO)
GROUPINGS = [((2,),(1,)), ((2,),(0,)), ((1,),(0,)), ((3,),(2,)), ((3,),(0,1,2)), ((2,1),(0,)), ((3,),(1,)),  ((3,),(0,))]

# Always use synthetic data
FORCE_SYNTHETIC = True

# ----------------------------
# Synthetic dataset generator (100 up + 100 down per class, unique)
# ----------------------------
def generate_synthetic_csv(csv_path: str,
                           n_classes: int = 4,
                           samples_per_class: int = 100,
                           n_noise_features: int = 10000,
                           effect_size: float = 1.0,
                           feature_sd: float = 1.0,
                           bin_value: int = 1,
                           random_state: int = 42):
    """
    Creates a CSV at csv_path with columns: 'bin','target', and feature columns f0000..fXXXX.
    For each class c, we set 200 unique marker features: 100 up-regulated, 100 down-regulated (non-overlapping across classes).
    """
    rng = np.random.default_rng(random_state)

    # Total marker features: 200 per class (100 up + 100 down), disjoint across classes
    markers_per_class = 200
    total_marker_features = markers_per_class * n_classes

    # Add some extra noise features
    D = total_marker_features + n_noise_features

    # Build disjoint marker index sets per class
    all_feat_indices = np.arange(D)
    class_markers = {}
    start = 0
    for c in range(n_classes):
        idxs = all_feat_indices[start:start+markers_per_class]
        start += markers_per_class
        up_idxs = idxs[:markers_per_class//2]    # first 100
        down_idxs = idxs[markers_per_class//2:]  # next 100
        class_markers[c] = (up_idxs, down_idxs)

    # Generate base noise for all samples
    N = samples_per_class * n_classes
    X = rng.normal(loc=0.0, scale=feature_sd, size=(N, D))

    # Apply class effects (unique markers per class)
    y = np.repeat(np.arange(n_classes), samples_per_class)
    for c in range(n_classes):
        idx = np.where(y == c)[0]
        up, down = class_markers[c]
        X[idx[:, None], up]  += effect_size
        X[idx[:, None], down] -= effect_size

    # Make features strictly positive (so later log2FC is stable after your scaling)
    X = np.exp(X / 2.0)  # log-normal-ish, > 0, preserves separations

    # Create DataFrame
    cols = [f"f{str(i).zfill(4)}" for i in range(D)]
    df = pd.DataFrame(X, columns=cols)
    df.insert(0, "target", y.astype(int))
    df.insert(0, "bin", bin_value)

    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    df.to_csv(csv_path, index=False)
    print(f"[SYNTH] Wrote synthetic dataset to: {csv_path}  | samples={N}, D={D}")
    print(f"[SYNTH] Per class markers: 100 up + 100 down (unique sets per class)")

# ----------------------------
# KFold+Repeats trainer
# ----------------------------
def train_kfold_repeats(X: np.ndarray, Y: np.ndarray, seed_base: int):
    kf = KFold(n_splits=K_SPLITS, shuffle=True, random_state=42)
    all_models = []
    num_classes = int(np.max(Y)) + 1
    for fold, (tr, va) in enumerate(kf.split(X, Y), 1):
        X_tr, y_tr = X[tr], Y[tr]
        X_va, y_va = X[va], Y[va]
        for r in range(N_REPEATS):
            seed = seed_base * 1000 + fold * 100 + r
            tf.keras.utils.set_random_seed(seed)
            np.random.seed(seed)
            m = build_model(X.shape[1], num_classes)
            m.fit(X_tr, y_tr, epochs=EPOCHS, batch_size=BATCH_SIZE,
                  validation_data=(X_va, y_va), verbose=0)
            all_models.append(m)
        print(f"[Seed base {seed_base}] Fold {fold}/{K_SPLITS} trained {N_REPEATS} models (total: {len(all_models)})")
    return all_models

# ----------------------------
# MAIN (bin-root outputs + keep only split CSVs)
# ----------------------------
def main():
    # --- Always generate synthetic dataset (100 up + 100 down unique per class) ---
    if FORCE_SYNTHETIC:
        generate_synthetic_csv(
            csv_path=CSV_PATH,
            n_classes=4,
            samples_per_class=100,   # 100 samples per class
            n_noise_features=10000,
            effect_size=1.0,
            feature_sd=1.0,
            bin_value=1,
            random_state=42
        )

    df = pd.read_csv(CSV_PATH)
    # Discover bins
    bins_found = sorted([b for b in df["bin"].dropna().unique().tolist()])
    if BIN_WHITELIST is not None:
        bins_to_process = [b for b in bins_found if b in set(BIN_WHITELIST)]
    else:
        bins_to_process = bins_found
    if len(bins_to_process) == 0:
        raise ValueError(f"No 'bin' values found in {CSV_PATH}")

    # Keep only classes 0..3 by default
    df = df[df["target"].astype(int).isin([0,1,2,3])].copy()
    unique_labels = np.sort(df["target"].astype(int).unique())
    assert unique_labels[0] == 0 and np.array_equal(unique_labels, np.arange(unique_labels[-1] + 1)), \
        f"Non-contiguous labels detected: {unique_labels}. Please remap to 0..C-1."
    groupings = GROUPINGS
    print(f"\nUsing fixed grouping(s): {groupings}")

    os.makedirs(OUT_ROOT, exist_ok=True)

    for BIN_VALUE in bins_to_process:
        print(f"\n================= BIN {BIN_VALUE} =================")
        models_A = models_B = None
        X = Y = fdf = None

        try:
            fdf = df[df["bin"] == BIN_VALUE].copy()
            if fdf.empty:
                print(f"[WARN] No rows for bin {BIN_VALUE}; skipping.")
                continue

            # Normalize all features except ['bin','target'] within this bin
            cols_to_norm = fdf.columns.difference(['bin', 'target'])
            fdf[cols_to_norm] = fdf[cols_to_norm].apply(lambda x: x / (x.max() + 1.0))

            Y = fdf["target"].astype(int).to_numpy()
            feature_cols = fdf.columns.difference(['bin', 'target'])
            X = np.nan_to_num(fdf[feature_cols].to_numpy(), copy=False).astype(np.float32)

            if X.shape[0] < 2 or X.shape[1] < 1:
                print(f"[WARN] Insufficient data for bin {BIN_VALUE} (samples={X.shape[0]}, dim={X.shape[1]}). Skipping.")
                continue

            print(f"Bin {BIN_VALUE}: samples={X.shape[0]}, dim={X.shape[1]}  class_counts="
                  f"{dict(zip(*np.unique(Y, return_counts=True)))}")

            # Output dirs per bin (everything under bin folder)
            OUT_DIR   = os.path.join(OUT_ROOT, f"bin_{str(BIN_VALUE).replace('.', '_')}")
            CSV_DIR   = os.path.join(OUT_DIR, "csv")
            PLOTS_DIR = os.path.join(OUT_DIR, "plots")
            os.makedirs(OUT_DIR, exist_ok=True)
            os.makedirs(CSV_DIR, exist_ok=True)
            os.makedirs(PLOTS_DIR, exist_ok=True)

            # Train ensembles ONCE per run (A, B)
            models_A = train_kfold_repeats(X, Y, seed_base=SEED_BASES[0])
            models_B = train_kfold_repeats(X, Y, seed_base=SEED_BASES[1])

            # grid helper (for your mirror plots)
            def _make_grid(n):
                n_grid = min(10000, n)
                x_grid = np.arange(600, 600 + 0.1 * n_grid, 0.1)[:n_grid]
                return n_grid, x_grid

            def save_compare_only(pos_ids, neg_ids):
                # ---- Gradients (Run A & B) — unchanged, NO TIC normalization
                grad_A = compute_avg_group_logodds_gradient(X, models_A, pos_ids=pos_ids, neg_ids=neg_ids, eps=1e-8)
                grad_B = compute_avg_group_logodds_gradient(X, models_B, pos_ids=pos_ids, neg_ids=neg_ids, eps=1e-8)
                grad_mean = (grad_A + grad_B) / 2.0

                pos_tag = "_".join(map(str, pos_ids))
                neg_tag = "_".join(map(str, neg_ids))
                tag = f"pos_{pos_tag}__neg_{neg_tag}"

                # ---- Mirror plots
                n_grid, x_grid = _make_grid(min(grad_A.size, grad_B.size))
                yA = grad_A[:n_grid]; yB = grad_B[:n_grid]
                yA_pos = np.where(yA > 0, yA, 0.0)
                yB_pos = np.where(yB > 0, yB, 0.0)
                yA_neg = np.where(yA < 0, -yA, 0.0)  # abs
                yB_neg = np.where(yB < 0, -yB, 0.0)

                cos_pos = cosine_sim(yA_pos, yB_pos)
                cos_neg = cosine_sim(yA_neg, yB_neg)

                comb_csv = os.path.join(CSV_DIR, f"grads_AB__{tag}.csv")
                pd.DataFrame({
                    "m/z": x_grid,
                    "grad_runA": yA,
                    "grad_runB": yB,
                    "pos_runA": yA_pos,
                    "pos_runB": yB_pos,
                    "negabs_runA": yA_neg,
                    "negabs_runB": yB_neg,
                }).to_csv(comb_csv, index=False)

                pos_title = (f"Bin {BIN_VALUE} — Mirror Positive Gradients "
                             f"[{tag}] (cos={cos_pos:.4f})")
                neg_title = (f"Bin {BIN_VALUE} — Mirror Negative Gradients |abs| "
                             f"[{tag}] (cos={cos_neg:.4f})")

                mirror_plot(
                    x_grid, yA_pos, yB_pos,
                    title=pos_title,
                    outfile=os.path.join(PLOTS_DIR, f"{tag}__mirror_pos.png")
                )
                mirror_plot(
                    x_grid, yA_neg, yB_neg,
                    title=neg_title,
                    outfile=os.path.join(PLOTS_DIR, f"{tag}__mirror_negabs.png")
                )

                # ---- Volcano statistics WITHOUT TIC normalization
                pvals, log2fc = ttest_and_log2fc(X, Y, pos_ids, neg_ids, eps=1e-12)

                # Save per-feature stats + raw grads + mean grad
                stats_csv = os.path.join(CSV_DIR, f"stats_and_grads__{tag}.csv")
                pd.DataFrame({
                    "feature": list(feature_cols),
                    "log2FC": log2fc,
                    "pval": pvals,
                    "neglog10p": -np.log10(np.clip(pvals, 1e-300, 1.0)),
                    "grad_runA_full": grad_A,
                    "grad_runB_full": grad_B,
                    "grad_mean": grad_mean,
                    "signed_stat": log2fc * (-np.log10(np.clip(pvals, 1e-300, 1.0))),
                }).to_csv(stats_csv, index=False)

                volcano_plot(
                    log2fc, pvals, LFC_LEFT, LFC_RIGHT, ALPHA,
                    title=f"Bin {BIN_VALUE} — Volcano ({tag})",
                    outfile=os.path.join(PLOTS_DIR, f"{tag}__volcano.png")
                )
                r = signed_agreement_plot(
                    log2fc, pvals, grad_mean,
                    LFC_LEFT, LFC_RIGHT, ALPHA,
                    title_prefix=f"Bin {BIN_VALUE} — Signed agreement ({tag})",
                    outfile=os.path.join(PLOTS_DIR, f"{tag}__signed_agreement.png")
                )

                # summary JSON directly under bin/
                with open(os.path.join(OUT_DIR, f"summary__{tag}.json"), "w") as fC:
                    json.dump({
                        "bin": BIN_VALUE,
                        "grouping": {"pos_ids": list(pos_ids), "neg_ids": list(neg_ids)},
                        "comparison": "Run A vs Run B",
                        "cosine_pos": cos_pos,
                        "cosine_neg_abs": cos_neg,
                        "pearson_signed_agreement": r,
                        "thresholds": {"alpha": ALPHA, "lfc_left": LFC_LEFT, "lfc_right": LFC_RIGHT},
                        "paths": {
                            "combined_csv_for_mirror": comb_csv,
                            "stats_and_grads_csv": stats_csv,
                            "plots_dir": PLOTS_DIR,
                        }
                    }, fC, indent=2)

                print(f"  [COMPARE] {tag} | Cos(pos)={cos_pos:.6f} Cos(neg|abs|)={cos_neg:.6f}  r={r:.3f}")

            # run all groupings -> write combined CSVs/plots/JSONs
            num_classes = int(np.max(Y)) + 1
            for (pos_ids, neg_ids) in groupings:
                for idx in (*pos_ids, *neg_ids):
                    assert 0 <= idx < num_classes, f"Class index {idx} out of range 0..{num_classes-1}"
                save_compare_only(pos_ids, neg_ids)

            # --- Split step: create split CSVs into csv/result/ ---
            split_outputs = process_folder(CSV_DIR, int(BIN_VALUE))

            # Move split files from csv/result/ -> csv/ and delete originals
            result_dir = os.path.join(CSV_DIR, "result")
            moved = []
            if os.path.isdir(result_dir):
                for fname in os.listdir(result_dir):
                    src = os.path.join(result_dir, fname)
                    dst = os.path.join(CSV_DIR, fname)
                    os.replace(src, dst)
                    moved.append(dst)
                try: os.rmdir(result_dir)
                except OSError: pass

            # Delete original combined CSVs (keep only split)
            for fname in os.listdir(CSV_DIR):
                if fname.lower().endswith(".csv") and fname.startswith("grads_AB__"):
                    try:
                        os.remove(os.path.join(CSV_DIR, fname))
                    except Exception as e:
                        print(f"[WARN] Could not remove {fname}: {e}")

            # Save manifest of final CSVs
            with open(os.path.join(OUT_DIR, "split_manifest.json"), "w") as fman:
                json.dump({
                    "bin": BIN_VALUE,
                    "final_csvs": sorted([os.path.basename(p) for p in moved]),
                }, fman, indent=2)

        finally:
            try:
                if models_A is not None:
                    for _m in models_A: del _m
                del models_A
            except: pass
            try:
                if models_B is not None:
                    for _m in models_B: del _m
                del models_B
            except: pass
            for v in ["X","Y","fdf"]:
                try: del globals()[v]
                except: pass
            hard_free()

    print("\nAll bins processed. (bin-root outputs; csv keeps only split files)\n")

# ---- run ----
if __name__ == "__main__":
    main()


[SYNTH] Wrote synthetic dataset to: F:/20251010/dataset_rt_batch.csv  | samples=400, D=10800
[SYNTH] Per class markers: 100 up + 100 down (unique sets per class)

Using fixed grouping(s): [((2,), (1,)), ((2,), (0,)), ((1,), (0,)), ((3,), (2,)), ((3,), (0, 1, 2)), ((2, 1), (0,)), ((3,), (1,)), ((3,), (0,))]

================= BIN 1 =================
Bin 1: samples=400, dim=10800  class_counts={0: 100, 1: 100, 2: 100, 3: 100}


c:\Users\benja\miniconda3\envs\vae\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[Seed base 111] Fold 1/5 trained 5 models (total: 5)
[Seed base 111] Fold 2/5 trained 5 models (total: 10)
[Seed base 111] Fold 3/5 trained 5 models (total: 15)
[Seed base 111] Fold 4/5 trained 5 models (total: 20)
[Seed base 111] Fold 5/5 trained 5 models (total: 25)
[Seed base 777] Fold 1/5 trained 5 models (total: 5)
[Seed base 777] Fold 2/5 trained 5 models (total: 10)
[Seed base 777] Fold 3/5 trained 5 models (total: 15)
[Seed base 777] Fold 4/5 trained 5 models (total: 20)
[Seed base 777] Fold 5/5 trained 5 models (total: 25)
  [COMPARE] pos_2__neg_1 | Cos(pos)=0.386884 Cos(neg|abs|)=0.323962  r=0.320
  [COMPARE] pos_2__neg_0 | Cos(pos)=0.303017 Cos(neg|abs|)=0.369628  r=0.314
  [COMPARE] pos_1__neg_0 | Cos(pos)=0.291641 Cos(neg|abs|)=0.440761  r=0.319
  [COMPARE] pos_3__neg_2 | Cos(pos)=0.459026 Cos(neg|abs|)=0.261516  r=0.317
  [COMPARE] pos_3__neg_0_1_2 | Cos(pos)=0.487418 Cos(neg|abs|)=0.256704  r=0.279
  [COMPARE] pos_2_1__neg_0 | Cos(pos)=0.289770 Cos(neg|abs|)=0.404246  r=

Realistic training dataset

In [ ]:
# simulate_ms1_smallmols_multiclass_0p1amu.py
# ---------------------------------------------------
# - Profile MS1 at 0.1 amu bins (100–1000 m/z)
# - 100 singly charged small molecules ([M+H]+, z=1) with averagine isotopes
# - Multi-class dataset: each class has its own up/down-regulated molecules
# - Outputs:
#     ms1_multiclass_train.csv            (rows = scans, columns = target + mz_*)
#     ms1_multiclass_meta.csv             (molecule params + which class up/down)
#     ms1_multiclass_labelmap.json        (index -> class label)
#     (optional) quick QC plots per class (means)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt
from pathlib import Path
import json

# -------------------------------
# Configuration (edit as needed)
# -------------------------------
SEED = 2026
OUT_DIR = Path("ms1_multiclass_out"); OUT_DIR.mkdir(exist_ok=True)

# Spectral & molecule parameters
N_MOLECULES = 100
MASS_RANGE = (150.0, 900.0)      # neutral masses (Da)
MZ_MIN, MZ_MAX = 100.0, 1000.0
STEP = 0.1                        # 0.1 amu bins
PROTON = 1.007276466812           # [M+H]+, z=1
RES_FWHM = 0.1                    # fixed peak FWHM ~0.1 Da at all m/z
BASELINE_NOISE = 20.0
DYN_RANGE = (3e3, 2e5)            # base intensity scale before jitter/FC

# Dataset design
N_CLASSES = 4                     # number of classes
N_PER_CLASS = 20                  # scans per class
JITTER_LOW, JITTER_HIGH = 0.8, 1.2  # per-scan molecule jitter (±20%)

# Regulation design per class
UP_PER_CLASS = 10                 # number of up-regulated molecules per class
DOWN_PER_CLASS = 10               # number of down-regulated molecules per class
UP_FC  = 5.0                      # fold-change for up-regulated
DOWN_FC = 0.2                     # fold-change for down-regulated

# Plot mean spectra per class?
MAKE_QC_PLOTS = True

# -------------------------------
# Helpers
# -------------------------------
rng = np.random.default_rng(SEED)
mz = np.arange(MZ_MIN, MZ_MAX + STEP, STEP)
POINTS = len(mz)

def gaussian(x, mu, fwhm=RES_FWHM):
    sigma = fwhm / 2.355
    return np.exp(-0.5 * ((x - mu) / sigma) ** 2)

def iso_env_small(mass):
    """Averagine-ish C-13 envelope (discrete Gaussian over isotope index k)."""
    C = (mass / 111.1254) * 4.9384
    p = 0.0109
    mean_k = p * C
    var_k = p * (1 - p) * C
    sd_k = sqrt(max(var_k, 1e-9))
    k = np.arange(0, int(mean_k + 6 * sd_k) + 1)
    env = np.exp(-0.5 * ((k - mean_k) / sd_k) ** 2)
    env /= env.sum() + 1e-12
    return k, env

# -------------------------------
# Define molecules (100)
# -------------------------------
# Spread base concentrations across 1→1000 so molecules have diverse intensities
concs = np.geomspace(1.0, 1000.0, N_MOLECULES)
rng.shuffle(concs)

molecules = []
for i in range(N_MOLECULES):
    mass = rng.uniform(*MASS_RANGE)
    apex_mz = mass + PROTON               # [M+H]+, z=1
    base_int = rng.uniform(*DYN_RANGE) * concs[i]
    k_vals, k_fracs = iso_env_small(mass)
    molecules.append({
        "molecule_id": i,
        "neutral_mass_Da": mass,
        "mz_apex": apex_mz,
        "k_vals": k_vals,
        "k_fracs": k_fracs,
        "base_intensity": base_int,
        "relative_conc": concs[i],
    })

base_scales = np.array([m["base_intensity"] for m in molecules])

# -------------------------------
# Build class-specific signatures
# -------------------------------
# For each class, pick UP_PER_CLASS and DOWN_PER_CLASS molecule IDs.
# We allow partial overlap across classes to make the problem realistic.
all_ids = np.arange(N_MOLECULES)
class_signatures = []
for c in range(N_CLASSES):
    up_ids = rng.choice(all_ids, size=UP_PER_CLASS, replace=False)
    remaining = np.setdiff1d(all_ids, up_ids)
    down_ids = rng.choice(remaining, size=DOWN_PER_CLASS, replace=False)
    class_signatures.append({"up": np.sort(up_ids), "down": np.sort(down_ids)})

# -------------------------------
# Scan simulator
# -------------------------------
def simulate_scan(intensity_scales: np.ndarray) -> np.ndarray:
    """Render one MS1 profile at 0.1 amu bins."""
    sig = np.zeros_like(mz)
    for i, m in enumerate(molecules):
        for k, frac in zip(m["k_vals"], m["k_fracs"]):
            pk_mz = m["mz_apex"] + k  # z=1 → +1.000 m/z per isotope
            if MZ_MIN <= pk_mz <= MZ_MAX:
                sig += intensity_scales[i] * frac * gaussian(mz, pk_mz)
    baseline = 80.0 + 20.0 * (mz - MZ_MIN) / (MZ_MAX - MZ_MIN)
    return np.clip(sig + rng.normal(0, BASELINE_NOISE, size=POINTS), 0, None)

# -------------------------------
# Generate scans for each class
# -------------------------------
rows = []
label_map = {i: f"class_{i}" for i in range(N_CLASSES)}  # customize labels here

for class_idx in range(N_CLASSES):
    sig = class_signatures[class_idx]
    up_mask = np.zeros(N_MOLECULES, dtype=bool);  up_mask[sig["up"]] = True
    dn_mask = np.zeros(N_MOLECULES, dtype=bool);  dn_mask[sig["down"]] = True

    for _ in range(N_PER_CLASS):
        scales = base_scales * rng.uniform(JITTER_LOW, JITTER_HIGH, N_MOLECULES)
        scales[up_mask] *= UP_FC
        scales[dn_mask] *= DOWN_FC
        spectrum = simulate_scan(scales)
        rows.append((class_idx, spectrum))

# -------------------------------
# Save training table (wide)
# -------------------------------
train_df = pd.DataFrame(
    [r[1] for r in rows],
    columns=[f"mz_{i}" for i in range(POINTS)]
)
train_df.insert(0, "target", [r[0] for r in rows])  # integer targets 0..N_CLASSES-1

train_path = OUT_DIR / "ms1_multiclass_train.csv"
train_df.to_csv(train_path, index=False)

# -------------------------------
# Save metadata: molecules + class signatures
# -------------------------------
meta_df = pd.DataFrame({
    "molecule_id": [m["molecule_id"] for m in molecules],
    "neutral_mass_Da": [m["neutral_mass_Da"] for m in molecules],
    "mz_apex": [m["mz_apex"] for m in molecules],
    "base_intensity": [m["base_intensity"] for m in molecules],
    "relative_conc": [m["relative_conc"] for m in molecules],
})
# Expand signatures into Boolean columns per class (up/down)
for c in range(N_CLASSES):
    up_col = np.isin(meta_df["molecule_id"], class_signatures[c]["up"])
    dn_col = np.isin(meta_df["molecule_id"], class_signatures[c]["down"])
    meta_df[f"class{c}_is_up"] = up_col
    meta_df[f"class{c}_is_down"] = dn_col

meta_path = OUT_DIR / "ms1_multiclass_meta.csv"
meta_df.to_csv(meta_path, index=False)

# Save label map and signatures for reproducibility
with open(OUT_DIR / "ms1_multiclass_labelmap.json", "w") as f:
    json.dump(label_map, f, indent=2)

with open(OUT_DIR / "ms1_multiclass_signatures.json", "w") as f:
    json.dump(
        {str(c): {"up": class_signatures[c]["up"].tolist(),
                  "down": class_signatures[c]["down"].tolist()}
         for c in range(N_CLASSES)},
        f, indent=2
    )

# -------------------------------
# Optional QC plots: mean spectra per class
# -------------------------------
if MAKE_QC_PLOTS:
    fig, ax = plt.subplots(figsize=(14, 5))
    cmap = plt.cm.get_cmap("tab10", N_CLASSES)
    for c in range(N_CLASSES):
        block = train_df.loc[train_df["target"] == c].drop(columns="target").values
        ax.plot(mz, block.mean(axis=0), lw=0.9, label=label_map[c], color=cmap(c))
    ax.set_title("Class-wise Mean MS1 Spectra (0.1 amu)")
    ax.set_xlabel("m/z"); ax.set_ylabel("Intensity (a.u.)")
    ax.legend(ncol=2)
    plt.tight_layout()
    plt.savefig(OUT_DIR / "ms1_multiclass_means.png", dpi=180)
    plt.close()

print("Saved:")
print(" -", train_path)
print(" -", meta_path)
print(" -", OUT_DIR / "ms1_multiclass_labelmap.json")
print(" -", OUT_DIR / "ms1_multiclass_signatures.json")
if MAKE_QC_PLOTS:
    print(" -", OUT_DIR / "ms1_multiclass_means.png")
print(f"Rows: {len(train_df):,}  |  Columns: {train_df.shape[1]:,}  (includes 'target')")
